# Lab 2: PID Control of a Mass-Spring-Damper System

Type name here

Monday 12:10pm section

Table #

## Contents:

1. [Pre-lab Questions](#prelab)
1. [Lab Procedure](#lab)
1. [Post-lab Questions](#postlab)
1. [Submissions](#sub)

## Learning Objectives

1. Implement P, PI, and PID control laws on mass-spring-damper systems
2. Computationally measure the closed-loop response characteristics assuming 2nd-order
3. Determine the effects of controller parameters on closed-loop response characteristics

<a id='prelab'></a>

## Pre-lab Questions

This lab concerns a simple, yet representative, mechanical system: the mass-spring-damper system shown below.  A free-body-diagram (FBD) of this system appears to the right.

<img src="https://github.com/AdaWick/MEEG312/blob/master/Lab1/mass%20spring%20damper.png?raw=true" width="600" />

The equation of motion (EOM) for this system derived using Newton's Second Law is as follows:

\\(m \ddot{x} = \sum F_x = -b \dot{x} - kx + F\\)

which can be rewritten as

\\(m \ddot{x} + b \dot{x} + kx = F\\)

If we divide this equation through by \\(m\\), then we arrive at the following *canonical form*:

\\(\ddot{x} + 2 \zeta \omega_n \dot{x} + \omega_n^2 = \dfrac{F}{m}\\)

where \\(\zeta\\) is the [damping ratio](https://en.wikipedia.org/wiki/Damping_ratio) and \\(\omega_n\\) is the 
[natural frequency](https://en.wikipedia.org/wiki/Simple_harmonic_motion#Dynamics).

In this lab, we will explore how different types of feedback affect the step response of the system.  (More information about the step response can be found in the notebook `Time Response.ipynb` on Canvas.)

The specifications we use to quantify step responses are shown in the following figure:

<img src="https://github.com/AdaWick/MEEG312/blob/master/Lab2/second%20order%20specs.png?raw=true" width="600" />

They are defined as follows:

* Rise time (\\(t_r\\)): The time taken for the output to go from 10% to 90% of the final value

* Peak time (\\(t_p\\)): The time taken for the output to attain its value

* Percent overshoot (\\(\%OS\\)): The maximum output relative to the final value given as a percentage, i.e. \\(\%OS = \dfrac{max value - final value}{final value} \times 100\%\\)

* Settling time (\\(t_s\\)): The time taken for the output to be bounded within \\(\pm 2\%\\) of its final value

* Steady state error (\\(e_{ss}\\)): The difference between the final value of the output and the input step value.

Now, let's load a data file like the one you will be acquiring during the lab.  Check out the file `test2.txt`.  In this lab, we will only use the columns 'Time' and 'Encoder 1 Pos'.  'Time' is a list of time stamps when the data is acquired.  The data in the rest of the columns is collected simultaneously at each time stamp.  'Encoder 1 Pos' is the output of the encoder that is used to measure the position of the cart.

The following code loads the text file using [numpy.genfromtext](https://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html), which is a generic function for reading in text files.  Because it only handles rows with consistent formatting, we have to ignore the first two rows and the last one.  We also have to ignore the `;` that appears at the end of each row.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as ss

In [ ]:
# read data from text file, skipping first 2 rows and last row, ignore ';'
data = np.genfromtxt('test2.txt',comments=';',skip_header=3,skip_footer=1)
t = data[:,1]     # time is column 2 (recall column numbering starts at 0)
x = data[:,3]     # position is column 4
plt.plot(t,x);

Notice that there are two steps: a step up to 1000 (encoder counts) and a step back down to 0.  In order to calculate the above quantities (rise time, peak time, etc.), we only need to analyze the first step response (the step up to 1000).  **You may assume an input step size of 1000 counts for calculating the steady-state error.**

**Problem 1 (15 points):**

Complete the following script to calculate each of the 5 specifications listed above.

Here are some functions that will be useful:

* [numpy.flatnonzero](https://numpy.org/doc/stable/reference/generated/numpy.flatnonzero.html): gives the indices of an array that are nonzero or true.  For example: `np.flatnonzero(x > .5)` would give the indices corresponding to points where $x > 0.5$, so you can use this to find where $x > 0.5$
* [numpy.argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html): gives the index where the maximum value of an array occurs.  For example `ind = np.argmax(x)` gives the index of when the max of $x$ occurs, and so then `t[ind]` would be the time at which the max occurs
* [numpy.argmin](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html): gives the index where the minimum value of an array occurs.  For example `ind = np.argmin(x)` gives the index of when the min of $x$ occurs, and so then `t[ind]` would be the time at which the min occurs

In [ ]:
# trim everything after 3 seconds
x = x[t < 3]
t = t[t < 3]

# rise time
t_r = 
print(f'Rise time = {t_r:g} s')

# peak time
t_p =
print(f'Peak time = {t_p:g} s')

# percent overshoot
OS = 
print(f'Percent overshoot = {OS:g} %')

# settling time
t_s = 
print(f'Settling time = {t_s:g} s')

# steady-state error
e_ss = 
print(f'Steady-state error = {e_ss:g}')

plt.plot(t,x);

For the last part of the pre-lab, we want to be able to use the peaks (local maxima) of the response to estimate the period of oscillation.  For this part, you can reuse your code from Lab 1.

To find the peaks, we will use the function [scipy.signal.find_peaks](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html).  You should try to average over a number of cycles to get a more accurate estimate of the period.

**Problem 2 (5 points):**

Use the `find_peaks` function to estimate the period of oscillation of the step response.

In [ ]:
from scipy.signal import find_peaks

period = 
print(f'Period = {period:g} s')

<a id='lab'></a>

## Lab Procedure

In this lab, you will explore the effects of [Proportional, Integral, Derivative feedback control](https://en.wikipedia.org/wiki/PID_controller).  Each of these terms will be explored separately, then in pairs, and finally all simultaneously.

Control problems usually involve making a system track a desired trajectory through space.  In this lab, we want the cart to track a step-up, step-down trajectory, which appears in the following figure:

In [ ]:
t = np.linspace(0,6,1001)
dwell_time = 3
amplitude = 1000
x = np.zeros_like(t)
x[t<dwell_time] = amplitude
x[0] = 0

plt.plot(t,x)
plt.ylabel('Encoder counts')
plt.xlabel('Time [s]');

**In-person labs:**

To set this function as the desired trajectory, follow these steps:

1. Open the ECP Executive software from the desktop icon. Go to *Command > Trajectory*, select *Step*, then click *Setup*.

2. Choose *Closed Loop Step* and set *step size* = 1000, *dwell time* = 3000, *number of reps* = 1. Click *OK* and close the window, then click *OK* again to close the next window.

**For this lab, the motion controller next to your computer must be turned on.**

<p style="border:3px; border-style:solid; border-color:#0000FF; padding: 1em;">
Each trial will proceed as follows:
<br><br>
1. Go to *Setup > Control algorithm* and set the radio button to *PID* (keep it here for the entire lab).
<br><br>
2. Click *Setup algorithm*.  In this window, input your desired values of \\(K_P\\), \\(K_I\\), and \\(K_D\\).  Click *OK*.
<br><br>
3. Click the *Implement algorithm*.  **This step is crucial for changing the gains of your controller**; if you forget this step, it will keep using the previous values.  Close the window by hitting *OK*.
<br><br>
4. Go to *Command > Execute* and click *Run*.  The system will run for about 6 seconds (double the *dwell time* you input earlier).
<br><br>
5. To see the response, go to *Plotting > Setup Plot*. Choose *Encoder 1 position* and *Commanded Position* only in the left axis box (you may need to remove some options that are already there), then click plot data. The plot will show the trajectory of the cart and the commanded position (a square wave).  After you setup the plot the first time, you only need to select *Plotting > Plot data* to see your plot.  If this trial is a "keeper", take a screenshot and save the image of the plot with an appropriate filename for potential inclusion in your post-lab analysis.
<br><br>
6. If you want to save the data, go to *Data > Export raw data*. Pick an appropriate file name, and export the data somewhere you can access it later (portable memory drive/USB stick, Google Drive, etc.).  If you run another experiment, you will lose the data from this one if you don't save it!
</p>

**Turn off the motion controller and close the ECP software after all trials are done.**

**Online labs**

Open the file "Lab 2 simulator.ipynp" in a separate tab.

1. In the third cell from the top, input the parameters for each experiment.  In particular, you will be changing the controller gains according the procedures of Parts 1-4 below.

2. Optionally save an image of the plot of the cart's motion.

3. Run the last cell to save your data into a text file.  It will prompt you for a file name.  You should not add the ".txt" to the file name; this will be added automatically.  **Be sure to do this after every simulation you want to save.  If you don't, your data will be overwritten.**

<p style="color:green; font-weight:bold; font-size:18px">The objective of this lab is to get the cart's response to match the desired trajectory (square wave) as closely as possible.  You will not be able to accomplish this until Part 4; however, note how the response gets better as you progress from Part 1 to Part 4.</p>

### Part 1: Proportional Control

Proportional control is the simplest form of control in terms of implementation and understanding its behavior.  The idea is that if the cart is not in the desired position, a force should be applied in the direction of the desired position.  Furthermore, the magnitude of the force should be proportional to how far away the cart is from the desired position.  So if the cart is close to where it should be, apply a small force, and if it is far away, apply a larger force.

First, we define the error as $e(t) = r(t) - x(t)$, where $r(t)$ is the reference position (the desired position), and $x(t)$ is the actual position of the cart at time $t$. 

Mathematically, the proportional control law is given by

$F(t) = K_P e(t)$

where $F(t)$ is the force the motor outputs on the cart, and $K_P$ is the **proportional gain** which is a design parameter that you set.

Set $K_I = K_D = 0$ for Part 1 of the lab.  You will only be varying $K_P$ in Part 1.

Now, gradually increase $K_P$ from an initial value of $K_P = 0.1$.  Do not exceed a value of $K_P = 0.5$ in Part 1 (you can exceed it in later parts.)

**Explain the behavior you see as you change $K_P$.  Try to state your observations in terms of the response specifications from the pre-lab.  Does the cart get better or worse at tracking the square wave?  Save 2-3 trials to be able to back up your statements.**

### Part 2: Proportional-Integral Control

The cart, like all mechanical systems, is subject to static friction.  This means that if the cart stops moving, it requires a certain force to get it moving again.  What if the cart stops close to its desired position?  Will the proportional control force be large enough to overcome static friction?

Integral control is specifically designed to overcome steady-state error.  If a small error is integrated over time, it becomes a larger and larger quantity.  Eventually, it will become large enough to overcome the static friction and get the cart moving again.

Mathematically, a proportional-integral controller can be summarized by

$F(t) = K_P e(t) + K_I \int_{\tau=0}^{\tau=t} e(\tau) d\tau$

First, set \\(K_P = 0.2, K_I = 0\\) and run a trial.  (Set \\(K_D = 0\\) for this part of the lab.)  Then change \\(K_I = 0.2\\) and keep \\(K_P\\) the same.  **Explain the difference in these results.**

(The integral control action may sometimes take a while to see.  In this case, increase the *dwell time* sufficiently for the system to reach steady state.)

**Explain the behavior you see as you change \\(K_I\\).  Try to state your observations in terms of the response specifications from the pre-lab.  Save 2-3 trials to be able to back up your statements.**

### Part 3: Proportional-Derivative Control

You probably noticed how oscillatory the cart's motion is under P and PI control.  Wouldn't it be better if it just moved directly to the desired position without oscillating around it?

From your studies of 2nd-order systems, you know that increasing the damping in the system can shift it from underdamped to overdamped, which has the effect of eliminating oscillations.  Derivative control is specifically designed to mimic the behavior of a linear damper/dashpot.

Mathematically, a proportional-derivative controller can be summarized by

$F(t) = K_P e(t) + K_D \dfrac{de(t)}{dt}$

Notice that the second term has the same mathematical form as a linear damper: \\(b \dfrac{dx(t)}{dt}\\)

Note that you would never use derivative control by itself, i.e. without a proportional term.  The derivative term cannot sense the position itself (only velocity), so it has no idea how close it is to the desired value.

First, recall the trial in which \\(K_P = 0.2, K_D = 0\\) (and \\(K_I = 0\\), which you will keep constant for this part of the lab.)  Then change \\(K_D = 0.001\\) and keep \\(K_P\\) the same.  Notice any difference?   

Next, try increasing the damping in stages up to \\(K_D = 0.02\\).  Try 2 or 3 different values of $K_D$.  **Explain the change in response you see as \\(K_D\\) increases.**

**Try to determine when the system becomes critically damped.  Do you think this is the "optimal" response?**

### Part 4: Proportional-Integral-Derivative Control

By now, you probably have a better feel for how each term individually affects the step response of the cart.

For the final part of the lab, set all three gains, \\(K_P, K_I, K_D\\), to obtain a "good" response of the system.  It is up to you to define what this means.

**Turn off the motion controller and close the ECP software after all trials are done.**

<a id='postlab'></a>

## Post-lab Questions

For each of the four sections of the lab procedure, you (hopefully) gathered enough data to show how changing the gains \\(K_P, K_I, K_D\\) of the controller affects its closed-loop step response.

Please submit a discussion of your results for each section.  You may embed your discussions within the appropriate sections of the lab precedure, or insert them below, making sure to label each sub-section.  Use as many plots and trials as needed to sufficiently support your discussion.  You should include plots showing how the various specs like settling time change as the gains of the PID controller are varied.

At the end, please write a conclusion paragraph (or two) summarizing your results and outlining the process for how a PID controller could be designed for this mass-spring-damper system.

Part 1 Discussion
![title](Screenshots/Part1_P.1.PNG)
![title](Screenshots/Part1_P.35.PNG)
![title](Screenshots/Part1_P.5.PNG)

In the three screenshots above, P = .1, .35, .5, respectively. The only system that was able to settle was the first, the other two had too much overshoot due to the high P, this casued an ocilating motion. It seems that as P goes up, the cart's tracking ability goes down.

Part 2 Discussion
![title](Screenshots/Part2_I.0.PNG)
![title](Screenshots/Part2_I.2.PNG)
![title](Screenshots/Part2_I.2_dwell=6s.PNG)

In the three screenshots above, P = .2 and I = 0, .2, .2, respectively. The last screenshot has an elongated dwell period of 6000 ms, compared to the typical 3000 ms. As I increases, the steady state error decreases. This is due to the I parameter trying to reduce the error over time.


Part 3 Discussion
![title](Screenshots/Part3_D.01.PNG)
![title](Screenshots/Part3_D.001.PNG)
![title](Screenshots/Part3_D.02.PNG)
![title](Screenshots/Part3_D.005.PNG)
![title](Screenshots/Part3_D.007.PNG)

In the 5 screenshots above, P = .2 and D = .01, .001, .02, .05, .07, respectively. The system becomes critically dampened around D = .007, where the displcement is immediately met and has little overshoot. The exact value for D is near .007. This is the optimal response for some things that only care about reaching the desired value as fast as possible. However, if this is used as a car's cruise control, the acceleration caused by the near instant jump would be uncomfortable and cause unnecesary damage to the engine. There are cases where this is optimal and others where it is not.


Part 4 Discussion
![title](Screenshots/Part4_P.8_D.015_I.5.PNG)

In the screenshot above, a PID controller is selected that does not have any overshoot, but still reaches the target value in an acceptable amount of time. P = .8, D = .015, I = .5.

<a id='sub'></a>

## Submissions

Please submit the following on Canvas:

1. Your completed Jupyter notebook (this file)
2. All data (.txt) files
3. All screen captures (they can be embedded in the Jupyter notebook or saved as separate files)

Please label your data files and screen captures in a logical manner so that they can be correlated.